In [ ]:
!git clone https://github.com/gunthercox/chatterbot-corpus.git

Cloning into 'chatterbot-corpus'...
remote: Enumerating objects: 2310, done.
remote: Total 2310 (delta 0), reused 0 (delta 0), pack-reused 2310 (from 1)
Receiving objects: 100% (2310/2310), 548.25 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (1374/1374), done.


In [ ]:
!pip install transformers pyyaml
!pip install pyyaml sentence-transformers scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
import os
import yaml
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Load and Parse YAML Files
def load_yaml_conversations(directory):
    conversations = []
    for filename in os.listdir(directory):
        if filename.endswith('.yaml') or filename.endswith('.yml'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf8') as file:
                data = yaml.safe_load(file)
                convs = data.get('conversations', [])
                for conv in convs:
                    # Handle both simple Q&A and multi-turn conversations
                    if len(conv) == 2:
                        conversations.append((conv[0].strip(), conv[1].strip()))
                    elif len(conv) > 2:
                        # For multi-turn, you can choose to concatenate or handle differently
                        for i in range(0, len(conv)-1, 2):
                            q = conv[i].strip()
                            a = conv[i+1].strip()
                            conversations.append((q, a))
    return conversations

# Step 2: Generate Embeddings
def generate_embeddings(conversations, model):
    questions = [q for q, a in conversations]
    embeddings = model.encode(questions, convert_to_tensor=True)
    return questions, embeddings

# Step 3: Find the Best Matching Answer
def find_best_match(user_input, questions, embeddings, model, top_k=1):
    user_embedding = model.encode([user_input], convert_to_tensor=True)
    cosine_scores = cosine_similarity(user_embedding.cpu().numpy(), embeddings.cpu().numpy())[0]
    top_results = np.argsort(cosine_scores)[::-1][:top_k]
    return top_results, cosine_scores[top_results]

# Step 4: Chatbot Interaction Loop
def chat_bot(conversations, questions, embeddings, model):
    print("Chatbot ready. Type 'exit' to stop.")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Bot: Goodbye!")
            break
        top_indices, scores = find_best_match(user_input, questions, embeddings, model)
        if scores[0] < 0.5:  # Threshold for similarity
            print("Bot: I'm not sure how to respond to that.")
        else:
            response = conversations[top_indices[0]][1]
            print(f"Bot: {response}")

def main():
    # Directory containing your YAML files
    yaml_directory = '/content/chatterbot-corpus/chatterbot_corpus/data/english/'  # Replace with your YAML files directory path

    # Load conversations
    conversations = load_yaml_conversations(yaml_directory)
    if not conversations:
        print("No conversations found. Please check your YAML files.")
        return

    # Initialize the sentence transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose other models as well

    # Generate embeddings for the questions
    questions, embeddings = generate_embeddings(conversations, model)

    # Start chatting
    chat_bot(conversations, questions, embeddings, model)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Chatbot ready. Type 'exit' to stop.
You: hai
Bot: I'm not sure how to respond to that.
You: exit
Bot: Goodbye!


In [ ]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import os
import yaml
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import streamlit as st

# Step 1: Load and Parse YAML Files
def load_yaml_conversations(directory):
    conversations = []
    for filename in os.listdir(directory):
        if filename.endswith('.yaml') or filename.endswith('.yml'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf8') as file:
                data = yaml.safe_load(file)
                convs = data.get('conversations', [])
                for conv in convs:
                    if len(conv) == 2:
                        conversations.append((conv[0].strip(), conv[1].strip()))
                    elif len(conv) > 2:
                        for i in range(0, len(conv)-1, 2):
                            q = conv[i].strip()
                            a = conv[i+1].strip()
                            conversations.append((q, a))
    return conversations

# Step 2: Generate Embeddings
def generate_embeddings(conversations, model):
    questions = [q for q, a in conversations]
    embeddings = model.encode(questions, convert_to_tensor=True)
    return questions, embeddings

# Step 3: Find the Best Matching Answer
def find_best_match(user_input, questions, embeddings, model, top_k=1):
    user_embedding = model.encode([user_input], convert_to_tensor=True)
    cosine_scores = cosine_similarity(user_embedding.cpu().numpy(), embeddings.cpu().numpy())[0]
    top_results = np.argsort(cosine_scores)[::-1][:top_k]
    return top_results, cosine_scores[top_results]

# Step 4: Streamlit App
def main():
    st.title("Chatbot Interface")

    # Directory containing your YAML files
    yaml_directory = '/content/chatterbot-corpus/chatterbot_corpus/data/english/'  # Replace with your YAML files directory path

    # Load conversations
    conversations = load_yaml_conversations(yaml_directory)
    if not conversations:
        st.error("No conversations found. Please check your YAML files.")
        return

    # Initialize the sentence transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose other models as well

    # Generate embeddings for the questions
    questions, embeddings = generate_embeddings(conversations, model)

    # User input
    user_input = st.text_input("You: ", "")

    if user_input:
        top_indices, scores = find_best_match(user_input, questions, embeddings, model)
        if scores[0] < 0.5:  # Threshold for similarity
            st.write("Bot: I'm not sure how to respond to that.")
        else:
            response = conversations[top_indices[0]][1]
            st.write(f"Bot: {response}")

if __name__ == "__main__":
    main()


2024-09-30 16:08:21.188 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 16:08:21.192 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-09-30 16:08:26.869 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 16:08:26.871 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-30 16:08:26.873 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when ru

In [ ]:
%%writefile app.py
import os
import yaml
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import streamlit as st

# Step 1: Load and Parse YAML Files
def load_yaml_conversations(directory):
    conversations = []
    for filename in os.listdir(directory):
        if filename.endswith('.yaml') or filename.endswith('.yml'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf8') as file:
                data = yaml.safe_load(file)
                convs = data.get('conversations', [])
                for conv in convs:
                    if len(conv) == 2:
                        conversations.append((conv[0].strip(), conv[1].strip()))
                    elif len(conv) > 2:
                        for i in range(0, len(conv)-1, 2):
                            q = conv[i].strip()
                            a = conv[i+1].strip()
                            conversations.append((q, a))
    return conversations

# Step 2: Generate Embeddings
def generate_embeddings(conversations, model):
    questions = [q for q, a in conversations]
    embeddings = model.encode(questions, convert_to_tensor=True)
    return questions, embeddings

# Step 3: Find the Best Matching Answer
def find_best_match(user_input, questions, embeddings, model, top_k=1):
    user_embedding = model.encode([user_input], convert_to_tensor=True)
    cosine_scores = cosine_similarity(user_embedding.cpu().numpy(), embeddings.cpu().numpy())[0]
    top_results = np.argsort(cosine_scores)[::-1][:top_k]
    return top_results, cosine_scores[top_results]

# Step 4: Streamlit App
def main():
    st.title("Chatbot Interface")

    # Directory containing your YAML files
    yaml_directory = '/content/chatterbot-corpus/chatterbot_corpus/data/english/'  # Replace with your YAML files directory path

    # Load conversations
    conversations = load_yaml_conversations(yaml_directory)
    if not conversations:
        st.error("No conversations found. Please check your YAML files.")
        return

    # Initialize the sentence transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose other models as well

    # Generate embeddings for the questions
    questions, embeddings = generate_embeddings(conversations, model)

    # User input
    user_input = st.text_input("You: ", "")

    if user_input:
        top_indices, scores = find_best_match(user_input, questions, embeddings, model)
        if scores[0] < 0.5:  # Threshold for similarity
            st.write("Bot: I'm not sure how to respond to that.")
        else:
            response = conversations[top_indices[0]][1]
            st.write(f"Bot: {response}")

if __name__ == "__main__":
    main()


Writing app.py


In [ ]:
!curl https://loca.lt/mytunnelpassword

35.221.235.237

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.221.235.237:8501

your url is: https://open-candies-follow.loca.lt
2024-09-30 16:15:21.502737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 16:15:21.542086: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 16:15:21.554694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 16:15:23.775914: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/